In [1]:
!ls ../data/*.h5ad

ls: cannot access '../data/*.h5ad': No such file or directory


In [2]:
!md5sum ../data/*.h5ad

md5sum: '../data/*.h5ad': No such file or directory


In [3]:
import numpy as np
import pandas as pd
import scanpy as sc

In [4]:
ad = sc.read_h5ad("../../data/MIBC_snSeq.h5ad")
ad

AnnData object with n_obs × n_vars = 67988 × 33538
    obs: 'subtype', 'initial_celltype', 'scVI_vanilla_leiden', 'celltype', 'hash_ID', 'batch', 'Patient'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'scVI_args'

In [5]:
ad.obs.subtype.value_counts()

CDH12_Epithelial      22296
KRT13_Epithelial      14643
Cycling_Epithelial    12939
UPK_Epithelial         5668
KRT6A_Epithelial       4224
Inflam_Macrophage      1497
Endothelial            1249
PDPN_Fibroblast        1079
Naive_Tcell             966
CD8T                    806
CD20_Bcell              793
PDGFRB_Fibroblast       475
FAP_Fibroblast          455
Treg                    339
Plasma_Bcell            224
MHCII_Macrophage        152
Dendritic_cell          116
ACTA2_Fibroblast         67
Name: subtype, dtype: int64

In [6]:
# Focus on these receptors
fibroblast_receptors = [ 'ITGA5', 'CD44', 'TGFBR1', 'CNTN1', 'NRP1', 'FAP', 'ITGA1', 'ITGA3',
                         'FLT1', 'PDGFRB', 'PLXND1', 'ITGA2', 'ITGB5', 'LRP6', 'EGFR', 'FGFR1',
                         'ERAP1', 'ROBO2', 'AXL', 'ABCA1', 'SMAD3', 'TGFBR2', 'ITGA8' ]
# tcell_receptors = [ 'ITGA1', 'CD38', 'TGFBR3', 'PTPRC', 'CXCR4', 'TGFBR2', 'SDC4', 'CD46', 'CD4' ]
combined_receptors = set(fibroblast_receptors)
with open('use_receptors.txt', 'w+') as f:
    for g in combined_receptors:
        f.write(f'{g}\n')

In [7]:
# Get ligands and associated genes that go with the focused receptors
LRtable = pd.read_csv("../cabello_aguilar_LR_PMID.csv", index_col=0, header=0)
LRtable = LRtable.loc[LRtable.receptor.isin(combined_receptors)]
ligands = set(LRtable.ligand)
print(len(ligands))

associated_genes = []
for g in combined_receptors:
    receptor_list = [l.strip() for l in open(f'../receptor_genelists_2020_06_30/{g}.txt', 'r')]
    associated_genes += receptor_list
    
associated_genes = set(associated_genes)
print(len(associated_genes))

192
1539


In [8]:
# subset genes from all of our focused lists
use_genes = combined_receptors.union(ligands).union(associated_genes)
print(len(use_genes))

ad = ad[:, ad.var_names.isin(use_genes)].copy()
ad

1682


AnnData object with n_obs × n_vars = 67988 × 1444
    obs: 'subtype', 'initial_celltype', 'scVI_vanilla_leiden', 'celltype', 'hash_ID', 'batch', 'Patient'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'scVI_args'

In [9]:
# sample cells with uniform sampling
np.random.seed(1337)

N = 500
use_cells = []
for c in np.unique(ad.obs.subtype):
    c_bc = ad.obs_names[ad.obs.subtype == c].tolist()
    c_bc = list(np.random.choice(c_bc, min(N, len(c_bc)), replace=False))
    use_cells += c_bc
print(len(use_cells))


7328


In [10]:
ad = ad[use_cells].copy()
ad

AnnData object with n_obs × n_vars = 7328 × 1444
    obs: 'subtype', 'initial_celltype', 'scVI_vanilla_leiden', 'celltype', 'hash_ID', 'batch', 'Patient'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'scVI_args'

In [11]:
ad.write('demo_GEX.h5ad')

In [12]:
!ls -lha *.h5ad
!md5sum *.h5ad

-rw-rw-r-- 1 ing ing  11M Nov 30 09:57 demo_GEX.h5ad
-rw-rw-r-- 1 ing ing 1.5M Nov 30 09:56 demo_receptor_scores.h5ad
4f731abe3b90ca20d3339dd01002df2b  demo_GEX.h5ad
4905a07d795754875fdaea444d2afde9  demo_receptor_scores.h5ad


Finally we need a directory with only the receptors of interest 

In [13]:
import shutil

source_dir = '../receptor_genelists_2020_06_30'
demo_dir = 'demo_receptor_genelists'

for r in combined_receptors:
    shutil.copyfile(f'{source_dir}/{r}.txt', f'{demo_dir}/{r}.txt')

In [14]:
!ls demo_receptor_genelists

ABCA1.txt  EGFR.txt   FLT1.txt	 ITGA5.txt  NRP1.txt	SMAD3.txt
AXL.txt    ERAP1.txt  ITGA1.txt  ITGA8.txt  PDGFRB.txt	TGFBR1.txt
CD44.txt   FAP.txt    ITGA2.txt  ITGB5.txt  PLXND1.txt	TGFBR2.txt
CNTN1.txt  FGFR1.txt  ITGA3.txt  LRP6.txt   ROBO2.txt
